In [1]:
from src.datamodule.gadme_datamodule import GADMEDataModule
from src.datamodule.base_datamodule import DatasetConfig, LoadersConfig, LoaderConfig
from src.datamodule.components.transforms import TransformsWrapper, PreprocessingConfig, BatchTransformer
from src.datamodule.components.event_mapping import XCEventMapping
from src.datamodule.components.event_decoding import EventDecoding
from src.datamodule.components.feature_extraction import DefaultFeatureExtractor
from omegaconf import DictConfig

import src.modules.models.embedding_models.perch_tf_embedding_model as embed
import src.modules.models.embedding_models.embedding_classifier_model as classifier

In [2]:
birdnet = embed.BirdNetTfEmbeddingModel()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
num_classes = 22
sample_rate = 48000
window_size_s = 3.0
learning_rate = 1e-3
num_epochs = 5

In [4]:
dataset_name = "DBD-research-group/gadme_v1"
cache_dir = "/Volumes/BigChongusF/Datasets/Huggingface/gadme_v1/data"
dataset_config = DatasetConfig(cache_dir, "high_sierras", dataset_name, "high_sierras", 42, num_classes, 3, 0.2, "multiclass", None, sample_rate)
loaders_config = LoadersConfig()
loaders_config.train = LoaderConfig(12, True, 6, True, False, True, 2)
loaders_config.valid = LoaderConfig(12, False)
loaders_config.test = LoaderConfig(12, False)
mapper = XCEventMapping(biggest_cluster=True,
                        event_limit=5,
                        no_call=True)
transforms_wrapper = BatchTransformer(
    task = "multiclass",
    sampling_rate=sample_rate,
    max_length=window_size_s)
dm = GADMEDataModule(dataset_config, loaders_config, transforms_wrapper, mapper)

In [5]:
dm.prepare_data()
dm.setup("fit")

Saving the dataset (0/1 shards):   0%|          | 0/21650 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5413 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10296 [00:00<?, ? examples/s]

In [6]:
from lightning import Trainer

In [7]:
model = classifier.TfEmbeddingClassifier(birdnet, num_classes, learning_rate, num_epochs)
model

1024


TfEmbeddingClassifier(
  (embedding_model): BirdNetTfEmbeddingModel()
  (linear): Linear(in_features=1024, out_features=22, bias=True)
  (m): Sigmoid()
  (criterion): CrossEntropyLoss()
  (acc): MulticlassAccuracy()
  (ece): MulticlassCalibrationError()
  (auroc): MulticlassAUROC()
)

In [8]:
loader = dm.train_dataloader()

In [9]:
trainer = Trainer(accelerator="cpu")

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
birdnet.embedding_dimension

1024

In [11]:
%set_env PYTORCH_ENABLE_MPS_FALLBACK=1

env: PYTORCH_ENABLE_MPS_FALLBACK=1


In [12]:
trainer.fit(model=model,
            datamodule=dm)


  | Name            | Type                       | Params
---------------------------------------------------------------
0 | embedding_model | BirdNetTfEmbeddingModel    | 0     
1 | linear          | Linear                     | 22.6 K
2 | m               | Sigmoid                    | 0     
3 | criterion       | CrossEntropyLoss           | 0     
4 | acc             | MulticlassAccuracy         | 0     
5 | ece             | MulticlassCalibrationError | 0     
6 | auroc           | MulticlassAUROC            | 0     
---------------------------------------------------------------
22.6 K    Trainable params
0         Non-trainable params
22.6 K    Total params
0.090     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]